In [7]:
# python env: atom2_env_dev_v2

import json                                                                                                                             
import monty.json
import numpy as np
import pandas as pd
import traceback as tb
import sys
 
from pymatgen.core.structure import Structure
 
from atomate2.vasp.jobs.core import DielectricMaker
from atomate2.vasp.powerups import (
    update_user_incar_settings,
    update_user_potcar_functional,
    update_user_kpoints_settings
)
 
from fireworks import LaunchPad
from importlib import reload
 
from jobflow import Flow
from jobflow import run_locally
from jobflow.managers.fireworks import flow_to_workflow
 
from pathlib import Path
import pickle
from tqdm import tqdm

# LaunchPad

In [14]:
#   Paths to the candidates datasets
candidate_df_path    = Path("df_selected_v1.json")
 
#   Loading the candidates datasets
if not candidate_df_path.exists():
    raise Exception(f"The file containing the structures of the candidates was not found at {candidate_df_path}.")
else:
    df_selected = pd.read_json(candidate_df_path)

#   Iterating over the candidates to add them in the Launchpad 
for mpid, d in tqdm(zip(df_selected.index.values, df_selected['structure'])):

    # Initializes the MP structure
    struc = Structure.from_dict(d)
    structure = struc.get_primitive_structure()
 
    # Create a Dfpt dielectric WF to compute the dielectric tensor
    dielec_flow = DielectricMaker().make(structure=structure)
 
    # Power ups
        # Increase the kpoints density to 1500 per reciprocal atom for balance accuracy/complexity
    dielec_flow = update_user_kpoints_settings( dielec_flow, {"grid_density": 1500})
 
    # Let's add a metadata to recover it easily from the MongoDb afterwards with {"spec._tasks.job.metadata.Label": "HSE_Etot(x)"}
    dielec_flow.update_metadata({"Batch": "re2fractive_hg_v1_bisbis", "mp-id": f"{mpid}"})
 
 
 
        # Specify:  the parallelization to optimize performance
        #           the electronic convergence criterion            (1E-5 eV otherwise by default),
        #           do not enforce a type of psp --> read in POTCAR (PBEsol otherwise by default)
        #           projection done in real space to improve efficiency (True otherwise)
    dielec_flow = update_user_incar_settings(   dielec_flow, {  "KPAR": 8,
                                                                "EDIFF": 1E-6,
                                                                "GGA": None,
                                                                "LREAL": 'False',
                                                                "ALGO": 'All',
                                                                "ISMEAR": 0,
                                                                "SIGMA": 0.03,
                                                                "LAECHG": 'False',
                                                                "LELF": 'False',
                                                                "LVTOT": 'False',
                                                                "LWAVE": 'False',
                                                                "PREC": 'Accurate',
                                                                "IBRION": -1,
                                                                "NSW": 0
                                                                })
 
        # Choose the type of PSP, here PBE_54
    dielec_flow = update_user_potcar_functional(dielec_flow, "PBE_54")
 
    # convert the flow to a fireworks WorkFlow object
    wf = flow_to_workflow(dielec_flow)

    # # # # submit the workflow to the FireWorks launchpad (requires a valid connection)
    # lpad = LaunchPad.auto_load()
    # lpad.add_wf(wf)
 
#   Final step: launch the calculations on a cluster using qlaunch 

0it [00:00, ?it/s]

2024-01-09 17:33:53,734 INFO Added a workflow. id_map: {-1801: 8114}
2024-01-09 17:33:53,767 INFO Added a workflow. id_map: {-1802: 8115}
2024-01-09 17:33:53,800 INFO Added a workflow. id_map: {-1803: 8116}


3it [00:00, 28.77it/s]

2024-01-09 17:33:53,831 INFO Added a workflow. id_map: {-1804: 8117}
2024-01-09 17:33:53,863 INFO Added a workflow. id_map: {-1805: 8118}
2024-01-09 17:33:53,904 INFO Added a workflow. id_map: {-1806: 8119}


6it [00:00, 28.62it/s]

...

196it [00:09, 23.18it/s]

2024-01-09 17:34:02,788 INFO Added a workflow. id_map: {-1997: 8310}
2024-01-09 17:34:02,846 INFO Added a workflow. id_map: {-1998: 8311}
2024-01-09 17:34:02,924 INFO Added a workflow. id_map: {-1999: 8312}


199it [00:09, 21.14it/s]

2024-01-09 17:34:02,962 INFO Added a workflow. id_map: {-2000: 8313}


200it [00:09, 21.58it/s]
